# ETL pipeline with Incremental Data Load

In [2]:
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import os

In [3]:
#get password from environment var
pwd = os.environ['PGPASS']
uid = os.environ['PGUID']
driver ="{SQL Server Native Client 11.0}"
server ="localhost"
database="AdventureWorksDW2019;"

In [4]:
#connect to database sql server
scr_conn= pyodbc.connect('DRIVER=' +driver+ ';SERVER=' +server+ ';Trusted_Connection=yes'';UID=' +uid+ ';PWD='+pwd)

In [5]:
#Destination Postgresql
engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:5432/AdventureWorks')

# Read Data

In [6]:
source = pd.read_sql_query("""SELECT TOP 12 CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus FROM AdventureWorksDW2019.dbo.DimCustomer;""",scr_conn)
source

C:\Users\syaha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus
0,11000,26,AW00011000,None,Jon,V,Yang,False,1971-10-06,M
1,11001,37,AW00011001,None,Eugene,L,Huang,False,1976-05-10,S
2,11002,31,AW00011002,None,Ruben,None,Torres,False,1971-02-09,M
3,11003,11,AW00011003,None,Christy,None,Zhu,False,1973-08-14,S
4,11004,19,AW00011004,None,Elizabeth,None,Johnson,False,1979-08-05,S
5,11005,22,AW00011005,None,Julio,None,Ruiz,False,1976-08-01,S
6,11006,8,AW00011006,None,Janet,G,Alvarez,False,1976-12-02,S
7,11007,40,AW00011007,None,Marco,None,Mehta,False,1969-11-06,M
8,11008,32,AW00011008,None,Rob,None,Verhoff,False,1975-07-04,S
9,11009,25,AW00011009,None,Shannon,C,Carlson,False,1969-09-29,S


# Create and Load initial Data to Target,read data target before

In [7]:
#save the data to destination as the initial load. on the first run we load all data
tbl_name ="stg_IncrementalLoadTest"
source.to_sql(tbl_name, engine, if_exists='replace', index=False)

12

In [8]:
#read target data into dataframe
target=pd.read_sql_query('select * from etl."stg_IncrementalLoadTest"', engine)
target

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus
0,11000,26,AW00011000,None,Jon,V,Yang,False,1971-10-06,M
1,11001,37,AW00011001,None,Eugene,L,Huang,False,1976-05-10,S
2,11002,31,AW00011002,None,Ruben,None,Torres,False,1971-02-09,M
3,11003,11,AW00011003,None,Christy,None,Zhu,False,1973-08-14,S
4,11004,19,AW00011004,None,Elizabeth,None,Johnson,False,1979-08-05,S
5,11005,22,AW00011005,None,Julio,None,Ruiz,False,1976-08-01,S
6,11006,8,AW00011006,None,Janet,G,Alvarez,False,1976-12-02,S
7,11007,40,AW00011007,None,Marco,None,Mehta,False,1969-11-06,M
8,11008,32,AW00011008,None,Rob,None,Verhoff,False,1975-07-04,S
9,11009,25,AW00011009,None,Shannon,C,Carlson,False,1969-09-29,S


read Update Source data

In [36]:
#update on of record of midlename
source = pd.read_sql_query(''' SELECT TOP 12 CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus
FROM AdventureWorksDW2019.dbo.DimCustomer; ''', scr_conn)
source

C:\Users\syaha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus
0,11000,26,AW00011000,None,Jon,V,Yang,False,1971-10-06,M
1,11001,37,AW00011001,None,Eugene,L,Huang,False,1976-05-10,S
2,11002,31,AW00011002,None,Ruben,None,Torres,False,1971-02-09,M
3,11003,11,AW00011003,None,Christy,None,Zhu,False,1973-08-14,S
4,11004,19,AW00011004,None,Elizabeth,None,Johnson,False,1979-08-05,S
5,11005,22,AW00011005,None,Julio,None,Ruiz,False,1976-08-01,S
6,11006,8,AW00011006,None,Janet,G,Alvarez,False,1976-12-02,S
7,11007,40,AW00011007,None,Marco,None,Mehta,False,1969-11-06,M
8,11008,32,AW00011008,None,Rob,None,Verhoff,False,1975-07-04,S
9,11009,25,AW00011009,None,Shannon,C,Carlson,False,1969-09-29,S


Update a Source record, serve as modified row

In [37]:
# update some record . I will update the middle name for customerKey :1100
source.loc[source.MiddleName=='G', ['MiddleName']]='Gina'
source

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus
0,11000,26,AW00011000,None,Jon,V,Yang,False,1971-10-06,M
1,11001,37,AW00011001,None,Eugene,L,Huang,False,1976-05-10,S
2,11002,31,AW00011002,None,Ruben,None,Torres,False,1971-02-09,M
3,11003,11,AW00011003,None,Christy,None,Zhu,False,1973-08-14,S
4,11004,19,AW00011004,None,Elizabeth,None,Johnson,False,1979-08-05,S
5,11005,22,AW00011005,None,Julio,None,Ruiz,False,1976-08-01,S
6,11006,8,AW00011006,None,Janet,Gina,Alvarez,False,1976-12-02,S
7,11007,40,AW00011007,None,Marco,None,Mehta,False,1969-11-06,M
8,11008,32,AW00011008,None,Rob,None,Verhoff,False,1975-07-04,S
9,11009,25,AW00011009,None,Shannon,C,Carlson,False,1969-09-29,S


In [38]:
#detect change in data
target.apply(tuple,1)
target

0     (11000, 26, AW00011000, None, Jon, V, Yang, Fa...
1     (11001, 37, AW00011001, None, Eugene, L, Huang...
2     (11002, 31, AW00011002, None, Ruben, None, Tor...
3     (11003, 11, AW00011003, None, Christy, None, Z...
4     (11004, 19, AW00011004, None, Elizabeth, None,...
5     (11005, 22, AW00011005, None, Julio, None, Rui...
6     (11006, 8, AW00011006, None, Janet, G, Alvarez...
7     (11007, 40, AW00011007, None, Marco, None, Meh...
8     (11008, 32, AW00011008, None, Rob, None, Verho...
9     (11009, 25, AW00011009, None, Shannon, C, Carl...
10    (11010, 22, AW00011010, None, Jacquelyn, C, Su...
11    (11011, 22, AW00011011, None, Curtis, None, Lu...
dtype: object

In [39]:
source.apply(tuple,1).isin(target.apply(tuple,1))

0      True
1      True
2      True
3      True
4      True
5      True
6     False
7      True
8      True
9      True
10     True
11     True
dtype: bool

In [43]:
#Detect change get rows that are not present in the target
change = source[~source.apply(tuple,1).isin(target.apply(tuple,1))]
change

,CustomerKey,GeographyKey,CustomerAlternateKey,Title,FirstName,MiddleName,LastName,NameStyle,BirthDate,MaritalStatus
6,11006,8,AW00011006,None,Janet,Gina,Alvarez,False,1976-12-02,S


In [49]:
#get modified rows 
modified = change[change.CustomerKey.isin(target.CustomerKey)]
modified

AttributeError: 'Series' object has no attribute 'CustomerKey'

In [56]:
# Get new records
inserts = change[~change.CustomerKey.isin(target.CustomerKey)]
inserts

AttributeError: 'Series' object has no attribute 'CustomerKey'